In [122]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re
import json
import geopy
from geopy import distance
from datetime import datetime, date
import importlib

In [123]:
# file naming variables
today = date.today()
latest_version = 'df_2023-02-26'
location = '/Users/Sim/Documents/Other/Programming/Personal Projects/property_valuation'

### Loading data

In [124]:
df=pd.read_csv(f'{location}/data/{latest_version}.csv')
len(df)

4458

In [142]:
df.head()

,Date,Price,NearStat,StatDist(miles),Prop_Type,Bedrooms,Bathrooms,Floor_Size,Outdoor,Portered,...,Outcode,Outcode2,Outcode3,Postcode,Centrality,North,Links,Date_Scraped,Avg_price,Month_Year
0,20211111.0,600000.0,Barbican Station,0.4,Apartment,1.0,1.0,NaN,0.0,0.0,...,EC1V,EC1,EC1V3,EC1V3NY,2.845931,1.0,NaN,2022-09-02,109.0,202209
1,20220826.0,450000.0,Angel Station,0.2,Apartment,1.0,1.0,NaN,0.0,0.0,...,EC1V,EC1,EC1V4,EC1V4LB,2.880314,1.0,NaN,2022-09-02,134.0,202209
2,20220318.0,600000.0,Battersea Power Underground Station,0.2,Flat,1.0,1.0,NaN,1.0,0.0,...,SW11,SW11,SW118,SW118EB,2.987774,0.0,NaN,2022-09-02,185.0,202209
3,20220826.0,650000.0,Chancery Lane Station,0.2,Flat,1.0,1.0,NaN,0.0,0.0,...,WC1V,WC1,WC1V7,WC1V7HH,1.418421,1.0,NaN,2022-09-02,NaN,202209
4,20220826.0,550000.0,Pimlico Station,0.2,Flat,2.0,1.0,NaN,0.0,0.0,...,SW1P,SW1,SW1P4,SW1P4NR,1.790709,1.0,NaN,2022-09-02,142.0,202209


### Adding Data

In [126]:
# First import the scraping functions
#from ipynb.fs.full.RightMove_Funcs import scrape_results_page
#from ipynb.fs.full.RightMove_Funcs import scrape_listings
%run RightMove_Funcs.ipynb

In [127]:
# First Scraper 
all_apartment_links2 = scrape_results_page()[1]
#all_prices = scrape_results_page()[2]
#all_id_no = scrape_results_page()[3]

In [128]:
# Second Scraper, scraping only properties listed since last scrape
most_recent = int(max(df['Date'])) +1

scraped = scrape_listings(date_from = most_recent, links=all_apartment_links2)

# Obtain variables
all_links,all_features, all_prices = scraped[1], scraped[2],scraped[3]
all_statname, all_statdist, all_outcodes = scraped[4], scraped[5], scraped[6]
all_postcodes, all_centralities, all_dates = scraped[7],scraped[8],scraped[9]

Code is 0% completed
Code is 5% completed
Code is 10% completed
Code is 15% completed
Code is 20% completed
Code is 25% completed
Code is 30% completed
Code is 35% completed
Code is 40% completed
Code is 45% completed
Code is 50% completed
Code is 54% completed
Code is 59% completed
Code is 64% completed
Code is 69% completed
Code is 74% completed
Code is 79% completed
Code is 84% completed
Code is 89% completed
Code is 94% completed
Code is 99% completed


In [132]:
len(all_dates)

342

### Wrangling data

In [133]:
# Creating floor size (sq. ft.) variable

floor_size =[]

for i in range(len(all_features)):
    for j in range(len(all_features[i])):
        if all_features[i][j].endswith('sq. ft.'):
            clean1 = int(re.sub("[^0-9]", "", all_features[i][j]))
            if len(str(clean1)) > 4:   # Fixes instances where floor size is a range (e.g. 439-1051 sq feet)
                clean1 = str(clean1)
                half = int(len(clean1)/2)
                clean1 = int( (int(clean1[0:half]) + int(clean1[half:len(clean1)]) ) / 2)
            elif clean1 > 3000:  # Remove floorsizes > 3000
                clean1 = np.nan
            floor_size.append(clean1)
    if len(floor_size) <= i: 
        floor_size.append(np.nan)

In [134]:
# Keywords and Floor size (different route)

units = ['Sq ft', 'sq ft', 'Sq Ft', 'SQ FT', 'SqFt', 'sqft', 'Sqft', 'Square Feet', 'square feet', 'Square feet', 
         'square foot', 'Square foot', 'Square Foot', 'Sq. Ft.', 'sq. ft.', 'Sq. ft.', 'Sq. Ft','Sq. ft','sq. ft']

floor_size2 = []

# Extracting keywords: porter/concierge/caretaker, balcony/garden/terrace 

porter_list = ['Porter','PORTER','porter', 'Concierge','CONCIERGE','concierge','Caretaker','caretaker','CARETAKER']
outdoor_list = ['Balcony','BALCONY','balcony', 'Terrace','TERRACE','terrace', 'Patio', 'PATIO', 'patio', 'Garden', 'GARDEN', 'garden']

porter = []
outdoor = []

for i in range(len(all_links)):
    # Progress tracker
    if i % (len(all_links)//20) == 0:
        percent = round(i*100/len(all_links))
        print(f'Code is {percent}% completed')

    if i==0:
        r= requests.get(all_links[i])
    else:
        r = ''
        while r == '':
            try:
                r = requests.get(all_links[i])
                break
            except:
                print(f'Connection refused by the server on page {i+1}... sleeping for 3 seconds')
                time.sleep(3)
                print("Was a nice sleep, now let me continue...")
                continue

    soup = BeautifulSoup(r.text, 'lxml')
    
    # floor size
    lst = (soup.find_all("li", class_="lIhZ24u1NHMa5Y6gDH90A"))
    text =[str(x).replace('<li class="lIhZ24u1NHMa5Y6gDH90A">', '').replace('</li>', '') 
     for x in lst]
    floor_size2.append(np.nan)
    for j in text:
        #print(f'entry {clean1}, next text is {j}')
        if any(k in j for k in units):
            try: 
                position = j.index('sq')
            except ValueError: 
                try:
                    position = j.index('Sq')
                except ValueError: 
                    position = j.index('SQ')
                except:
                    pass
            str1 = j[0:position]
            clean1 = int(re.sub("[^0-9]", "", str1))
            if clean1 >= 3000 or clean1 <= 200:
                clean1 = np.nan
            floor_size2[i] = clean1
        else:
            clean1 = np.nan

    # keywords
    lst = (soup.find_all("li", class_="lIhZ24u1NHMa5Y6gDH90A"))
    text =[str(x).replace('<li class="lIhZ24u1NHMa5Y6gDH90A">', '').replace('</li>', '') 
     for x in lst]
    porter.append(0)
    outdoor.append(0)
    for j in text:
        if any(k in j for k in porter_list) and porter[i] < 1:
            porter[i] +=1
        if any(k in j for k in outdoor_list) and outdoor[i] < 1:
            outdoor[i] +=1

Code is 0% completed
Code is 5% completed
Connection refused by the server on page 31... sleeping for 3 seconds
Was a nice sleep, now let me continue...
Code is 10% completed
Code is 15% completed
Code is 20% completed
Code is 25% completed
Code is 30% completed
Code is 35% completed
Code is 40% completed
Code is 45% completed
Code is 50% completed
Code is 55% completed
Code is 60% completed
Code is 65% completed
Code is 70% completed
Code is 75% completed
Code is 80% completed
Code is 85% completed
Code is 89% completed
Code is 94% completed
Code is 99% completed


In [135]:
# Merge floor size lists
floor_size3 =  [None] * len(floor_size)
for i in range(len(floor_size)):
    if np.isnan(floor_size[i]) == True:
        floor_size3[i] = floor_size2[i]
    else:
        floor_size3[i] = floor_size[i]

In [136]:
# Creating property type variable
prop_type =[]

for i in all_features:
    if i[0] in ('Apartment','Flat','Studio','Maisonette', 'House','Terraced','Mews'):
        prop_type.append(i[0])
    else: 
        prop_type.append('')

In [137]:
# Creating bedrooms & bathrooms variable
bedrooms =[]
bathrooms =[]

import numpy as np

for i in all_features:
    if len(i)==1:
        bathrooms.append(np.nan)
        bedrooms.append(np.nan)
    elif i[0] == 'Studio':
        bedrooms.append(0)
        if i[1].startswith('×'):
            bathrooms.append(int(i[1][1]))
        else:
            bathrooms.append(np.nan)
    elif i[0].startswith('×') and i[1].startswith('×'): 
        bedrooms.append(int(i[0][1]))
        bathrooms.append(int(i[1][1]))
    elif len(i) <= 2:
        bedrooms.append(np.nan)
        bathrooms.append(np.nan)  
    elif i[1].startswith('×') and i[2].startswith('×'): 
        bedrooms.append(int(i[1][1]))
        bathrooms.append(int(i[2][1]))  
    else:
        bedrooms.append(np.nan)
        bathrooms.append(np.nan)

In [138]:
# Extracting keywords: porter/concierge/caretaker, balcony/garden/terrace 

porter = []
outdoor = []
porter_list = ['Porter','PORTER','porter', 'Concierge','CONCIERGE','concierge','Caretaker','caretaker','CARETAKER']
outdoor_list = ['Balcony','BALCONY','balcony', 'Terrace','TERRACE','terrace', 'Patio', 'PATIO', 'patio', 'Garden', 'GARDEN', 'garden']

for i in range(len(all_links)):
    
    # Progress tracker
    if i % (len(all_links)//20) == 0:
        percent = round(i*100/len(all_links))
        print(f'Code is {percent}% completed')
    
    r= requests.get(all_links[i])

    soup = BeautifulSoup(r.text, 'lxml')
    
    # keywords
    lst = (soup.find_all("li", class_="lIhZ24u1NHMa5Y6gDH90A"))
    text =[str(x).replace('<li class="lIhZ24u1NHMa5Y6gDH90A">', '').replace('</li>', '') 
     for x in lst]
    porter.append(0)
    outdoor.append(0)
    for j in text:
        if any(k in j for k in porter_list) and porter[i] < 1:
            porter[i] +=1
        if any(k in j for k in outdoor_list) and outdoor[i] < 1:
            outdoor[i] +=1

Code is 0% completed
Code is 5% completed
Code is 10% completed
Code is 15% completed
Code is 20% completed
Code is 25% completed
Code is 30% completed
Code is 35% completed
Code is 40% completed
Code is 45% completed
Code is 50% completed
Code is 55% completed
Code is 60% completed
Code is 65% completed
Code is 70% completed
Code is 75% completed
Code is 80% completed
Code is 85% completed
Code is 89% completed
Code is 94% completed
Code is 99% completed


In [139]:
# Creating contract type variable

contract =[]

for i in all_features:
    if i[-1] == 'Leasehold':
        contract.append('Leasehold')
    elif i[-1] == 'Share of Freehold':
        contract.append('Freehold')
    else:
        contract.append('')

In [140]:
# Adding new outcode variables

outcode2 =[]
for i in all_outcodes:
    if i[-1].isalpha() == True:
        outcode2.append(i[0:-1])
    else:
        outcode2.append(i)
        
outcode3 =[]
for i in all_postcodes:
    outcode3.append(i[:-2])

In [141]:
# North vs South variable
south = ['SE1','SE5','SE11','SE16','SE15','SE17','SW8','SW11','SW9','SW4','SW2']
north =[]
for i in outcode2:
    if i in south:
        north.append(0)
    else:
        north.append(1)

In [143]:
# date scraped and month_year
all_sc_dates = [str(today)] * len(all_links)
month_year = [int(i[0:4] + i[5:7]) for i in all_sc_dates]

In [147]:
# convert to dataframe
data = {"Date": all_dates,
        "Price": all_prices,
        "NearStat": all_statname,
        "StatDist(miles)": all_statdist,
        "Prop_Type": prop_type,
        "Bedrooms": bedrooms,
        "Bathrooms": bathrooms,
        "Floor_Size": floor_size3,
        "Outdoor": outdoor,
        "Portered": porter,
        "Contract_Type": contract,
        "Outcode": all_outcodes,
        "Outcode2": outcode2,
        "Outcode3": outcode3,
        "Postcode": all_postcodes,
        "Centrality": all_centralities,
        "North": north,
        "Links": all_links,
        "Date_Scraped": all_sc_dates,
        "Month_Year": month_year
       }
df2 = pd.DataFrame.from_dict(data)

In [148]:
# Check for no dataframe overlap (listings duplicated across both dataframes)
counter = 0
for i in range(len(df['Links'])):
    for j in range(len(df2['Links'])):
        if df.iloc[i,1] == df2.iloc[j,0]:
            counter +=1
            print(f'Overlap at {i} and {j}')
print(f'total overlap = {counter}')

total overlap = 0


In [149]:
# Remove any rows with price less than 10k
df2 = df2.drop(df2[df2.Price < 10000].index)
len(df2)

### Adding area data

In [200]:
# New area prices scrape if necessary
#area_scrape(year_month = 202304)

# Load in area data
area_df = pd.read_csv(f'{location}/data/area_df_2023.csv')
area_df = area_df[['Outcode3','Avg_price']]
len(area_df)
#area_df.head()

,Outcode3,Avg_price
0,BR11,52.5
1,BR12,120.0
2,BR13,74.0
3,BR14,67.0
4,BR15,62.5


In [214]:
df2 = df2.merge(area_df, on='Outcode3', how='left')

In [215]:
df2.tail()

,Date,Price,NearStat,StatDist(miles),Prop_Type,Bedrooms,Bathrooms,Floor_Size,Outdoor,Portered,...,Outcode,Outcode2,Outcode3,Postcode,Centrality,North,Links,Date_Scraped,Month_Year,Avg_price
337,20230301,450000,Marylebone Station,0.2,Apartment,1.0,1.0,NaN,0,1,...,NW1,NW1,NW16,NW16BT,3.041261,1,https://www.rightmove.co.uk/properties/1321365...,2023-03-11,202303,121.5
338,20230310,500000,Essex Road Station,0.2,Flat,1.0,1.0,NaN,0,0,...,N1,N1,N12,N12BE,3.954953,1,https://www.rightmove.co.uk/properties/1324418...,2023-03-11,202303,123.0
339,20230301,795000,Gloucester Road Station,0.2,Flat,1.0,1.0,NaN,0,0,...,SW7,SW7,SW74,SW74NQ,4.199528,1,https://www.rightmove.co.uk/properties/1321240...,2023-03-11,202303,249.0
340,20230308,800000,Angel Station,0.5,Apartment,1.0,1.0,855.0,0,0,...,N1,N1,N17,N17SH,3.534409,1,https://www.rightmove.co.uk/properties/1323826...,2023-03-11,202303,122.0
341,20230310,695000,Chalk Farm Station,0.3,Apartment,2.0,1.0,676.0,0,0,...,NW3,NW3,NW32,NW32BE,4.882361,1,https://www.rightmove.co.uk/properties/1324646...,2023-03-11,202303,177.0


### Joining datasets

In [207]:
# Drop previous avg_price column in case it has old numbers
df = df.drop(columns=['Avg_price'])

# Re-attach most up to date avg prices
df = df.merge(area_df, on='Outcode3', how='left')

In [216]:
df.tail()

,Date,Price,NearStat,StatDist(miles),Prop_Type,Bedrooms,Bathrooms,Floor_Size,Outdoor,Portered,...,Outcode,Outcode2,Outcode3,Postcode,Centrality,North,Links,Date_Scraped,Month_Year,Avg_price
4453,20230124.0,700000.0,Pimlico Station,0.3,Apartment,1.0,1.0,470.0,0.0,1.0,...,SW1V,SW1,SW1V2,SW1V2DN,2.143243,1.0,https://www.rightmove.co.uk/properties/1310270...,2023-02-26,202302,140.0
4454,20230124.0,600000.0,Kings Cross Thameslink Station,0.4,Apartment,2.0,1.0,641.0,0.0,1.0,...,N1,N1,N10,N10GL,3.337697,1.0,https://www.rightmove.co.uk/properties/1310252...,2023-02-26,202302,146.5
4455,20230124.0,650000.0,Chancery Lane Station,0.2,Apartment,1.0,1.0,441.0,1.0,0.0,...,WC1V,WC1,WC1V7,WC1V7HH,1.414989,1.0,https://www.rightmove.co.uk/properties/1310250...,2023-02-26,202302,91.0
4456,20230124.0,650000.0,Essex Road Station,0.4,Flat,2.0,1.0,NaN,1.0,0.0,...,N1,N1,N13,N13NY,4.781023,1.0,https://www.rightmove.co.uk/properties/1310249...,2023-02-26,202302,103.5
4457,20230124.0,575000.0,Finchley Road Station,0.2,Flat,2.0,1.0,NaN,1.0,0.0,...,NW6,NW6,NW63,NW63QR,5.860471,1.0,https://www.rightmove.co.uk/properties/1310247...,2023-02-26,202302,142.5


In [217]:
# Insert code to append newly scraped data to original df
df3 = df.append(df2)
len(df3)

/var/folders/xr/66lxz3_s6fg65z7bpjgp_yc80000gn/T/ipykernel_24420/2515260840.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df.append(df2)


4800

In [218]:
# Save to csv
today = date.today()
df3.to_csv(f'{location}/data/df_{today}.csv', index = False)